In [19]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime

In [20]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [21]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

(7875, 11)

In [23]:
# recuperar todos los pacientes ID
results = sf.query_all(
    """
  select Id, IdentificationId__pc,Name,poliza__c, Identification_Type__c, IsPersonAccount, Ocupaci_n__c, Pa_s_del_Paciente__c, PersonTitle ,PersonDepartment  from Account where ispersonaccount = True
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name",'poliza__c','Identification_Type__c','IsPersonAccount','Ocupaci_n__c','Pa_s_del_Paciente__c','PersonTitle','PersonDepartment']]
Pacientes_SF.shape

(1534418, 10)

In [24]:
# recuperar todos los pacientes ID
results = sf.query_all(
    """
   select Id,Name,IndividualId  from Contact
    """
)
Pacientesc_SF = pd.DataFrame(results["records"])

Pacientesc_SF = Pacientesc_SF[["Id", "IndividualId", "Name"]]
Pacientesc_SF.shape

(1534418, 3)

In [25]:
#Relacion Account Contact Relacion
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

(1220291, 5)

In [26]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId,ContactId,NameAccount__c,RutContact__c,Id_y,IndividualId,Name
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,None,Maldonado Velasquez Nicolas
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,None,Zaror Yunis Daniel Esteban
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,None,Bustamante Morales Flor Del Carmen
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,None,Malleo Abarca Rigoberto Andres
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,0038c000036jLDoAAM,None,Albanez Gonzalez Matias Oliver Franci


In [27]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId,ContactId,NameAccount__c,RutContact__c,Id_y,IndividualId,Name_x,Id,Name_y,IdentificationId__pc,Identification_Type__c,Holding__c,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,None,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,None,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,None,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,None,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,0038c000036jLDoAAM,None,Albanez Gonzalez Matias Oliver Franci,0018c00002axQa5AAE,Metlife Holding Metlife Finning,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5951,None,None


In [29]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_18104\3766479298.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Name_x'} in the result is deprecated and will raise a MergeError in a future version.
  Union_account_contact_relation = pd.merge(


,Id_x,AccountId,ContactId,NameAccount__c,RutContact__c,Id_y,IndividualId,Name_x,Id_x,Name_y,...,Id,IdentificationId__pc,Name_y,poliza__c,Identification_Type__c,IsPersonAccount_y,Ocupaci_n__c_y,Pa_s_del_Paciente__c,PersonTitle_y,PersonDepartment_y
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,None,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWElAAO,182204846,Maldonado Velasquez Nicolas,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,None,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amMfoAAE,18396206K,Zaror Yunis Daniel Esteban,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,None,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amRHeAAM,184262452,Bustamante Morales Flor Del Carmen,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,None,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWEmAAO,184429772,Malleo Abarca Rigoberto Andres,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,0038c000036jLDoAAM,None,Albanez Gonzalez Matias Oliver Franci,0018c00002axQa5AAE,Metlife Holding Metlife Finning,...,0018c00002cVCmMAAW,233986291,Albanez Gonzalez Matias Oliver Franci,None,RUT,True,None,a1U8c000005dAtnEAE,None,None


In [33]:
nuevos = "F:/Empresas SF/carga/sacarid.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
if ("rut1" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
    # aca se trabaja con el nuevo rut desde rut1 ojooooooo
    Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
    Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
    Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()

Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"]
)


Final_pacientes_y_empresa = Pacientes_Archivo

Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel("F:/Empresas SF/carga/pacientes_q_no_estan.xlsx", index=False)

pacientes_y_datos = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
pacientes_y_datos.to_excel("F:/Empresas SF/carga/pacientes_con_datos.xlsx", index=False)

In [ ]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])